## CIFAR 10 Object Recognition Project

In [121]:
#Import Libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.preprocessing import OneHotEncoder, Normalizer, LabelEncoder
from tqdm import tqdm
from skimage.feature import hog
from skimage import color
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score


In [68]:
# Load Data

current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)

#Define Train Path
train_path = os.path.join(base_dir, "cifar10/train")
test_path = os.path.join(base_dir, "cifar10/test")
train_path


'C:\\Users\\USER-PC\\Desktop\\Programming\\PYTHON\\cifar10_object_recognition\\cifar10/train'

In [69]:
# Preprocessing

Image_Size = 32



In [70]:
def preprocess_image(path,Image_Size):
    '''
    Input:  Path -> Image Path
            Image_Size -> intended size for each images

    Return: images, labels

    Preprocess the image into arrays
    
    '''

    images = []
    labels = []
    for class_name in tqdm(os.listdir(path)):
        class_path = os.path.join(path, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                
                img_path = os.path.join(class_path, img_file)
                
    
                
                # Resize Images
                img = cv2.imread(img_path)
                
                if img is not None:
                    img = cv2.resize(img, (Image_Size, Image_Size))
                    images.append(img)
                    labels.append(class_name)

    return images,labels
    
        
        

In [71]:
train_images, train_labels = preprocess_image(train_path,Image_Size)
test_images, test_labels = preprocess_image(test_path,Image_Size)


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:04<00:00, 18.46s/it]


In [96]:
# Convert to arrays
train_images = np.array(train_images, dtype="float32") / 255.0 
train_labels = np.array(train_labels)



In [97]:
# Convert to arrays
test_images = np.array(test_images, dtype="float32") / 255.0 
test_labels = np.array(test_labels)


### Feature Extraction (HOG)

In [108]:
train_images_hog = []

for img in tqdm(train_images): 
    gray_img = color.rgb2gray(img)   
    features = hog(gray_img, orientations=9, 
                   pixels_per_cell=(8, 8), 
                   cells_per_block=(2, 2), 
                   block_norm='L2-Hys')
    train_images_hog.append(features)

train_images_hog = np.array(train_images_hog)

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:21<00:00, 2284.51it/s]


In [84]:
#Save array as dataframe
train_images_df = pd.DataFrame({"image":list(train_images),
                                "labels": train_labels} )

test_images_df = pd.DataFrame({"image":list(test_images),
                                "labels": test_labels} )

,image,labels
0,"[[[0.58431375, 0.6117647, 0.60784316], [0.7333...",airplane
1,"[[[0.58431375, 0.4862745, 0.3529412], [0.58431...",airplane
2,"[[[0.9607843, 0.9882353, 0.96862745], [0.94509...",airplane
3,"[[[0.8745098, 0.94509804, 0.827451], [0.909803...",airplane
4,"[[[0.9843137, 0.52156866, 0.18431373], [0.9764...",airplane


In [88]:
#Save to csv

#define path
train_data_path = os.path.join(base_dir, 'cifar10/train_cifar10.csv')
test_data_path = os.path.join(base_dir, 'cifar10/test_cifar10.csv')

#Save to Path
train_images_df.to_csv(train_data_path, index=False)
test_images_df.to_csv(test_data_path, index=False)


In [102]:
#Encoding

encoder = LabelEncoder()
encoded_train_labels = encoder.fit_transform(train_labels)

print("Classes", encoder.classes_)
print("Labels", set(encoded_train_labels))

Classes ['airplane' 'automobile' 'bird' 'cat' 'deer' 'dog' 'frog' 'horse' 'ship'
 'truck']
Labels {np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9)}


## CLASSIFIERS
* Build two classifiers train one on the direct arrays and train one withe the extracted feature HOG

In [129]:
# Build classifier without HOG

train_images_resize = train_images.reshape(train_images.shape[0], -1)

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(
    train_images_resize, encoded_train_labels, test_size=0.2, random_state=42, stratify=encoded_train_labels
)

X_train_hog, X_val_hog, y_train_hog, y_val_hog = train_test_split(
    train_images_hog, encoded_train_labels, test_size=0.2, random_state=42, stratify=encoded_train_labels
)




In [127]:
log_reg = LogisticRegression(
    max_iter=1000, solver="saga", n_jobs=-1
)
log_reg.fit(X_train, y_train)

y_pred_lr = log_reg.predict(X_val)
acc_lr = accuracy_score(y_val, y_pred_lr)
print("Logistic Regression Accuracy:", acc_lr)


Logistic Regression Accuracy: 0.3152


In [128]:
print("\n📊 Logistic Regression Results")
print("Accuracy:", accuracy_score(y_val, y_pred_lr))
print("F1 Score (macro):", f1_score(y_val, y_pred_lr, average="macro"))
print("Classification Report:\n", classification_report(y_val, y_pred_lr, target_names=encoder.classes_))


📊 Logistic Regression Results
Accuracy: 0.3152
F1 Score (macro): 0.29207420672170475
Classification Report:
               precision    recall  f1-score   support

    airplane       0.32      0.46      0.38      1000
  automobile       0.37      0.25      0.30      1000
        bird       0.35      0.10      0.15      1000
         cat       0.27      0.07      0.11      1000
        deer       0.33      0.20      0.25      1000
         dog       0.29      0.39      0.33      1000
        frog       0.26      0.51      0.34      1000
       horse       0.30      0.19      0.24      1000
        ship       0.37      0.47      0.41      1000
       truck       0.33      0.51      0.40      1000

    accuracy                           0.32     10000
   macro avg       0.32      0.32      0.29     10000
weighted avg       0.32      0.32      0.29     10000



In [130]:
log_reg_hog = LogisticRegression(
    max_iter=1000, solver="saga", n_jobs=-1
)
log_reg_hog.fit(X_train_hog, y_train_hog)

y_pred_lr_hog = log_reg_hog.predict(X_val_hog)
acc_lr_hog = accuracy_score(y_val_hog, y_pred_lr_hog)
print("Logistic Regression Accuracy:", acc_lr_hog)

Logistic Regression Accuracy: 0.5137


In [131]:
print("\n📊 Logistic Regression Results")
print("Accuracy:", accuracy_score(y_val_hog, y_pred_lr_hog))
print("F1 Score (macro):", f1_score(y_val_hog, y_pred_lr_hog, average="macro"))
print("Classification Report:\n", classification_report(y_val_hog, y_pred_lr_hog, target_names=encoder.classes_))


📊 Logistic Regression Results
Accuracy: 0.5137
F1 Score (macro): 0.5099492796758632
Classification Report:
               precision    recall  f1-score   support

    airplane       0.62      0.58      0.60      1000
  automobile       0.60      0.63      0.61      1000
        bird       0.42      0.37      0.39      1000
         cat       0.39      0.29      0.33      1000
        deer       0.43      0.45      0.44      1000
         dog       0.42      0.44      0.43      1000
        frog       0.51      0.62      0.56      1000
       horse       0.54      0.55      0.54      1000
        ship       0.57      0.57      0.57      1000
       truck       0.61      0.65      0.63      1000

    accuracy                           0.51     10000
   macro avg       0.51      0.51      0.51     10000
weighted avg       0.51      0.51      0.51     10000



### 📊 Logistic Regression Results Without HOG vs With HOG
* Accuracy: 0.3152 -> 0.5137
* F1 Score (macro): 0.29207420672170475 | 0.5099492796758632
* Classification Report:
-               precision    recall  f1-score   support
-
-       airplane       0.32      0.46      0.38      1000  |   airplane       0.62      0.58      0.60      1000
-     automobile       0.37      0.25      0.30      1000  | automobile       0.60      0.63      0.61      1000
-           bird       0.35      0.10      0.15      1000  |       bird       0.42      0.37      0.39      1000
-            cat       0.27      0.07      0.11      1000 |         cat       0.39      0.29      0.33      1000
-           deer       0.33      0.20      0.25      1000 |        deer       0.43      0.45      0.44      1000
-            dog       0.29      0.39      0.33      1000 |         dog       0.42      0.44      0.43      1000
-           frog       0.26      0.51      0.34      1000 |        frog       0.51      0.62      0.56      1000
-          horse       0.30      0.19      0.24      1000 |       horse       0.54      0.55      0.54      1000
-           ship       0.37      0.47      0.41      1000 |        ship       0.57      0.57      0.57      1000
-          truck       0.33      0.51      0.40      1000 |       truck       0.61      0.65      0.63      1000

-       accuracy                           0.32     10000 |    accuracy                           0.51     10000
-      macro avg       0.32      0.32      0.29     10000 |   macro avg       0.51      0.51      0.51     10000
-     weighted avg     0.32      0.32      0.29     10000 |weighted avg       0.51      0.51      0.51     10000
